In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score

# --- CONFIGURATION ---
DATA_DIR = "/kaggle/input/new-dataset-for-efffnet/Final Dataset - Cleaned (1)"
OUTPUT_DIR = "/kaggle/working/"
IMG_SIZE = (224, 224)  # Paper specifies 224x224
BATCH_SIZE = 32
VAL_SPLIT = 0.2        # 80/20 split as recommended
EPOCHS_HEAD = 10       # Phase 1: Train head
EPOCHS_FINE = 15       # Phase 2: Fine-tune all
NUM_CLASSES = 124



2025-11-29 12:24:09.438648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764419049.659521      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764419049.722690      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
!pip install "protobuf==3.20.3" --quiet
print('installed')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [4]:
# --- 1. DATA AUGMENTATION (Paper Implementation) ---
# The paper explicitly uses Random Cropping, Horizontal Flipping, and Vertical Flipping
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,   # Simulates random cropping when combined with zoom
    height_shift_range=0.2,
    zoom_range=0.2,          # Simulates random cropping
    horizontal_flip=True,
    vertical_flip=True,      # Explicitly mentioned in Alam et al. (2025)
    fill_mode='nearest',
    validation_split=VAL_SPLIT
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VAL_SPLIT
)

# Load Data
print("Loading data...")
train_flow = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=42
)

val_flow = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=42,
    shuffle=False
)



Loading data...
Found 14051 images belonging to 124 classes.
Found 3449 images belonging to 124 classes.


In [6]:
# --- 2. CLASS WEIGHTS (Handle Imbalance) ---
class_weights_arr = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_flow.classes),
    y=train_flow.classes
)
class_weights_dict = dict(enumerate(class_weights_arr))
print(f"Class weights computed for {len(class_weights_dict)} classes.")



Class weights computed for 124 classes.


In [7]:
# --- 3. BUILD MODEL (EfficientNetB0) ---
def build_model(num_classes):
    # Load EfficientNetB0 pre-trained on ImageNet
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )

    # Freeze base model initially (Phase 1)
    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    
    # Paper uses a standard classifier head approach
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model, base_model

model, base_model = build_model(NUM_CLASSES)

# Metrics: Top-1 and Top-5 Accuracy
metrics_list = [
    'accuracy',
    tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')
]


I0000 00:00:1764419111.111675      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764419111.112295      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# --- 4. PHASE 1: TRAIN HEAD ONLY ---
print("\n--- Phase 1: Training Classifier Head (Frozen Backbone) ---")
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=metrics_list
)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(OUTPUT_DIR, "best_model_phase1.h5"),
        monitor='val_accuracy', save_best_only=True, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, restore_best_weights=True, verbose=1
    )
]

history_phase1 = model.fit(
    train_flow,
    epochs=EPOCHS_HEAD,
    validation_data=val_flow,
    class_weight=class_weights_dict,
    callbacks=callbacks
)



--- Phase 1: Training Classifier Head (Frozen Backbone) ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1764412894.394967     129 service.cc:148] XLA service 0x7be590002be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764412894.395818     129 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764412894.395842     129 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764412896.357881     129 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/440 ━━━━━━━━━━━━━━━━━━━━ 22s 52ms/step - accuracy: 0.0078 - loss: 6.9412 - top_5_accuracy: 0.0312       

...

440/440 ━━━━━━━━━━━━━━━━━━━━ 335s 762ms/step - accuracy: 0.6908 - loss: 1.0224 - top_5_accuracy: 0.9174 - val_accuracy: 0.7150 - val_loss: 1.0865 - val_top_5_accuracy: 0.9159
Epoch 4/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 668ms/step - accuracy: 0.7207 - loss: 0.8937 - top_5_accuracy: 0.9368
Epoch 4: val_accuracy improved from 0.71499 to 0.72050, saving model to /kaggle/working/best_model_phase1.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 338s 769ms/step - accuracy: 0.7207 - loss: 0.8938 - top_5_accuracy: 0.9368 - val_accuracy: 0.7205 - val_loss: 1.0907 - val_top_5_accuracy: 0.9133
Epoch 5/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.7510 - loss: 0.8030 - top_5_accuracy: 0.9489
Epoch 5: val_accuracy did not improve from 0.72050
440/440 ━━━━━━━━━━━━━━━━━━━━ 336s 764ms/step - accuracy: 0.7509 - loss: 0.8030 - top_5_accuracy: 0.9488 - val_accuracy: 0.7202 - val_loss: 1.0960 - val_top_5_accuracy: 0.9162
Epoch 6/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 666ms/step - accuracy: 0.7573 - loss: 0.7509 - top_5_accuracy: 0.9493
Epoch 6: val_accuracy improved from 0.72050 to 0.74224, saving model to /kaggle/working/best_model_phase1.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 337s 766ms/step - accuracy: 0.7573 - loss: 0.7510 - top_5_accuracy: 0.9492 - val_accuracy: 0.7422 - val_loss: 1.0556 - val_top_5_accuracy: 0.9243
Epoch 7/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 667ms/step - accuracy: 0.7688 - loss: 0.7151 - top_5_accuracy: 0.9563
Epoch 7: val_accuracy did not improve from 0.74224
440/440 ━━━━━━━━━━━━━━━━━━━━ 336s 764ms/step - accuracy: 0.7688 - loss: 0.7152 - top_5_accuracy: 0.9563 - val_accuracy: 0.7376 - val_loss: 1.0385 - val_top_5_accuracy: 0.9255
Epoch 8/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 660ms/step - accuracy: 0.7736 - loss: 0.6931 - top_5_accuracy: 0.9568
Epoch 8: val_accuracy did not improve from 0.74224
440/440 ━━━━━━━━━━━━━━━━━━━━ 333s 758ms/step - accuracy: 0.7736 - loss: 0.6932 - top_5_accuracy: 0.9568 - val_accuracy: 0.7399 - val_loss: 1.0528 - val_top_5_accuracy: 0.9290
Epoch 9/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 662ms/step - accuracy: 0.7815 - loss: 0.6800 - top_5_accuracy: 0.9624
Epoch 9: val_accuracy improved fro

440/440 ━━━━━━━━━━━━━━━━━━━━ 333s 757ms/step - accuracy: 0.7815 - loss: 0.6800 - top_5_accuracy: 0.9624 - val_accuracy: 0.7446 - val_loss: 1.0573 - val_top_5_accuracy: 0.9220
Epoch 10/10
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 662ms/step - accuracy: 0.7903 - loss: 0.6513 - top_5_accuracy: 0.9636
Epoch 10: val_accuracy improved from 0.74456 to 0.74775, saving model to /kaggle/working/best_model_phase1.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 334s 760ms/step - accuracy: 0.7903 - loss: 0.6514 - top_5_accuracy: 0.9636 - val_accuracy: 0.7478 - val_loss: 1.0394 - val_top_5_accuracy: 0.9301
Restoring model weights from the end of the best epoch: 10.


In [9]:
# --- 5. PHASE 2: FINE-TUNE ENTIRE NETWORK ---
# The paper highlights fine-tuning the *entire network* after head stabilization
print("\n--- Phase 2: Fine-Tuning Entire Network ---")

base_model.trainable = True # Unfreeze all layers

# Recompile with very low learning rate to prevent destroying learned weights
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5), # Low LR is critical here
    loss='categorical_crossentropy',
    metrics=metrics_list
)

callbacks_ft = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(OUTPUT_DIR, "best_model_final.h5"),
        monitor='val_accuracy', save_best_only=True, verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1
    )
]

history_phase2 = model.fit(
    train_flow,
    epochs=EPOCHS_FINE,
    validation_data=val_flow,
    class_weight=class_weights_dict,
    callbacks=callbacks_ft
)


--- Phase 2: Fine-Tuning Entire Network ---
Epoch 1/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 754ms/step - accuracy: 0.1103 - loss: 4.1584 - top_5_accuracy: 0.2751
Epoch 1: val_accuracy improved from -inf to 0.23253, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 450s 878ms/step - accuracy: 0.1103 - loss: 4.1581 - top_5_accuracy: 0.2752 - val_accuracy: 0.2325 - val_loss: 3.5391 - val_top_5_accuracy: 0.4494 - learning_rate: 1.0000e-05
Epoch 2/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.1704 - loss: 3.6798 - top_5_accuracy: 0.3940
Epoch 2: val_accuracy improved from 0.23253 to 0.31110, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 357s 812ms/step - accuracy: 0.1705 - loss: 3.6796 - top_5_accuracy: 0.3941 - val_accuracy: 0.3111 - val_loss: 3.0950 - val_top_5_accuracy: 0.5544 - learning_rate: 1.0000e-05
Epoch 3/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 736ms/step - accuracy: 0.2430 - loss: 3.2297 - top_5_accuracy: 0.4956
Epoch 3: val_accuracy improved from 0.31110 to 0.36590, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 367s 835ms/step - accuracy: 0.2430 - loss: 3.2295 - top_5_accuracy: 0.4957 - val_accuracy: 0.3659 - val_loss: 2.7352 - val_top_5_accuracy: 0.6367 - learning_rate: 1.0000e-05
Epoch 4/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.3012 - loss: 2.8820 - top_5_accuracy: 0.5758
Epoch 4: val_accuracy improved from 0.36590 to 0.41867, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 345s 785ms/step - accuracy: 0.3012 - loss: 2.8819 - top_5_accuracy: 0.5759 - val_accuracy: 0.4187 - val_loss: 2.4451 - val_top_5_accuracy: 0.7025 - learning_rate: 1.0000e-05
Epoch 5/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 692ms/step - accuracy: 0.3459 - loss: 2.6022 - top_5_accuracy: 0.6427
Epoch 5: val_accuracy improved from 0.41867 to 0.46999, saving model to /kaggle/working/best_model_final.h5


...

440/440 ━━━━━━━━━━━━━━━━━━━━ 340s 773ms/step - accuracy: 0.5673 - loss: 1.5781 - top_5_accuracy: 0.8469 - val_accuracy: 0.6347 - val_loss: 1.4427 - val_top_5_accuracy: 0.8637 - learning_rate: 1.0000e-05
Epoch 12/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 667ms/step - accuracy: 0.5924 - loss: 1.4981 - top_5_accuracy: 0.8560
Epoch 12: val_accuracy improved from 0.63468 to 0.64888, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 339s 770ms/step - accuracy: 0.5924 - loss: 1.4981 - top_5_accuracy: 0.8560 - val_accuracy: 0.6489 - val_loss: 1.3778 - val_top_5_accuracy: 0.8698 - learning_rate: 1.0000e-05
Epoch 13/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 677ms/step - accuracy: 0.6103 - loss: 1.3858 - top_5_accuracy: 0.8724
Epoch 13: val_accuracy improved from 0.64888 to 0.65729, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 341s 776ms/step - accuracy: 0.6103 - loss: 1.3859 - top_5_accuracy: 0.8724 - val_accuracy: 0.6573 - val_loss: 1.3169 - val_top_5_accuracy: 0.8791 - learning_rate: 1.0000e-05
Epoch 14/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 680ms/step - accuracy: 0.6259 - loss: 1.3679 - top_5_accuracy: 0.8792
Epoch 14: val_accuracy improved from 0.65729 to 0.67150, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 347s 788ms/step - accuracy: 0.6259 - loss: 1.3679 - top_5_accuracy: 0.8793 - val_accuracy: 0.6715 - val_loss: 1.2713 - val_top_5_accuracy: 0.8875 - learning_rate: 1.0000e-05
Epoch 15/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 723ms/step - accuracy: 0.6401 - loss: 1.2749 - top_5_accuracy: 0.8933
Epoch 15: val_accuracy improved from 0.67150 to 0.68136, saving model to /kaggle/working/best_model_final.h5


440/440 ━━━━━━━━━━━━━━━━━━━━ 364s 826ms/step - accuracy: 0.6401 - loss: 1.2749 - top_5_accuracy: 0.8933 - val_accuracy: 0.6814 - val_loss: 1.2321 - val_top_5_accuracy: 0.8916 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 15.


In [10]:
# --- 6. EVALUATION ---
print("\n--- Final Evaluation ---")
# Load the absolute best model from training
if os.path.exists(os.path.join(OUTPUT_DIR, "best_model_final.h5")):
    model.load_weights(os.path.join(OUTPUT_DIR, "best_model_final.h5"))

results = model.evaluate(val_flow)
print(f"Final Validation Accuracy: {results[1]*100:.2f}%")
print(f"Final Top-5 Accuracy: {results[2]*100:.2f}%")

# Save in format for inference
model.save(os.path.join(OUTPUT_DIR, "efficientnet_food_model.h5"))
print("Model saved successfully.")


--- Final Evaluation ---
108/108 ━━━━━━━━━━━━━━━━━━━━ 43s 397ms/step - accuracy: 0.6649 - loss: 1.2737 - top_5_accuracy: 0.8903


Final Validation Accuracy: 68.14%
Final Top-5 Accuracy: 89.16%
Model saved successfully.
